In [6]:
import plotly.express as px
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import CalculatedFieldSubroutines as cfs

In [25]:
gmID = "3a2a78cc-db21-11ee-a158-97f8443fd730"
#gmID = "2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4"

In [26]:
chassis_df = pd.read_csv(f"./data/chassis/{gmID}.csv")
best_pose_df = pd.read_csv(f"./data/best_pose/{gmID}.csv")

chassis_df = chassis_df.sort_values(by="time")
best_pose_df = best_pose_df.sort_values(by="time")

#cfs.BinaryDrivingMode(chassis_df)
#cfs.TernaryDrivingModeTransition(chassis_df)
#cfs.LatLonTotalStdDev(best_pose_df)
cfs.ChassisBestPoseMatchedTime(chassis_df,best_pose_df)


merged_df = pd.merge(chassis_df,best_pose_df,on="ChassisBestPoseMatchedTime")

#merged_df['time_sec'] = merged_df['time_x'].astype('datetime64[ns]')
#merged_df['time_sec'] = merged_df['time_sec'].astype('datetime64[s]')
#
#best_pose_df['time_sec'] = best_pose_df['time'].astype('datetime64[ns]')
#best_pose_df['time_sec'] = best_pose_df['time_sec'].astype('datetime64[s]')
#
#chassis_df['time_sec'] = chassis_df['time'].astype('datetime64[ns]')
#chassis_df['time_sec'] = chassis_df['time_sec'].astype('datetime64[s]')

#merged_df["drivingMode"] = np.where(merged_df["drivingMode"]  == "EMERGENCY_MODE", "COMPLETE_MANUAL",merged_df["drivingMode"])


merged_df


,yawRate,drivingMode,topic,msgsize,time_x,gearLocation,metadataID,throttlePercentage,steeringPercentage,brakePercentage,...,groupMetadataID_y,numSatsTracked,latitudeStdDev,differentialAge,longitude,solType,solutionAge,solStatus,latitude,numSatsInSolution
0,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692285265243057842,GEAR_PARKING,3a2a78cd-db21-11ee-a158-97f8443fd730,0.0,-6.307542,0.0,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.010128,1.0,-82.127926,NARROW_INT,0.0,SOL_COMPUTED,39.329602,15
1,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692285265274984780,GEAR_PARKING,3a2a78cd-db21-11ee-a158-97f8443fd730,0.0,-6.307542,0.0,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.010128,1.0,-82.127926,NARROW_INT,0.0,SOL_COMPUTED,39.329602,15
2,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692285265313385869,GEAR_PARKING,3a2a78cd-db21-11ee-a158-97f8443fd730,0.0,-6.307542,0.0,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.010128,1.0,-82.127926,NARROW_INT,0.0,SOL_COMPUTED,39.329602,15
3,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692285265341027973,GEAR_PARKING,3a2a78cd-db21-11ee-a158-97f8443fd730,0.0,-6.307542,0.0,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.010128,1.0,-82.127926,NARROW_INT,0.0,SOL_COMPUTED,39.329602,15
4,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692285265376207966,GEAR_PARKING,3a2a78cd-db21-11ee-a158-97f8443fd730,0.0,-6.307542,0.0,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.010128,1.0,-82.127926,NARROW_INT,0.0,SOL_COMPUTED,39.329602,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31898,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692287267717287214,GEAR_DRIVE,c9760136-db2a-11ee-a158-97f8443fd730,0.0,0.714985,26.9,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.009949,1.0,-82.128211,NARROW_INT,0.0,SOL_COMPUTED,39.330517,14
31899,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692287267749098870,GEAR_DRIVE,c9760136-db2a-11ee-a158-97f8443fd730,0.0,0.714985,26.9,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.009949,1.0,-82.128211,NARROW_INT,0.0,SOL_COMPUTED,39.330517,14
31900,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692287267783135265,GEAR_DRIVE,c9760136-db2a-11ee-a158-97f8443fd730,0.0,0.714985,26.9,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.009949,1.0,-82.128211,NARROW_INT,0.0,SOL_COMPUTED,39.330517,14
31901,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,1692287267814949035,GEAR_DRIVE,c9760136-db2a-11ee-a158-97f8443fd730,0.0,0.714985,26.9,...,3a2a78cc-db21-11ee-a158-97f8443fd730,15,0.009949,1.0,-82.128211,NARROW_INT,0.0,SOL_COMPUTED,39.330517,14


In [27]:
# Define your map with your dataframe name first and keep everything else the same
fig = px.scatter_mapbox(best_pose_df, 
                        lat="latitude",
                        lon="longitude",
                        hover_data=["time"],
                        #color="drivingMode",
                        zoom=13,
                        # color_discrete_map={
                        #   "M->A": "#63ACBE",
                        #   "A->M": "#601A4A",
                        #   "ERROR": "#EE442F",
                        #   "NONE":"yellowgreen",
                        # }
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }
                        #color_discrete_map={
                        #    "0": "#EE442F",
                        #    "1": "#63ACBE",
                        #}

                        )

# Layout settings
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.update_traces(marker=dict(size=8,opacity=1))

# Display the map
fig.show()

# Define your map with your dataframe name first and keep everything else the same
fig2 = px.scatter_mapbox(merged_df, 
                        lat="latitude",
                        lon="longitude",
                        hover_data=["time_x"],
                        #color="drivingMode",
                        zoom=13,
                        # color_discrete_map={
                        #   "M->A": "#63ACBE",
                        #   "A->M": "#601A4A",
                        #   "ERROR": "#EE442F",
                        #   "NONE":"yellowgreen",
                        # }
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }
                        #color_discrete_map={
                        #    "0": "#EE442F",
                        #    "1": "#63ACBE",
                        #}

                        )

# Layout settings
fig2.update_layout(mapbox_style="open-street-map")
fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig2.update_traces(marker=dict(size=8,opacity=1))

# Display the map
fig2.show()

In [10]:
d2_cols = [
    "drivingMode",
    "BinaryDrivingMode",
    "TernaryDrivingModeTransition",
    "time_x",
    "time_sec",
    "speedMps",
    "throttlePercentage",
    "brakePercentage",
    "steeringPercentage",
    "steeringRate",
    "numSatsInSolution",
    "LatLonTotalStdDev",
    "latitude",
    "longitude",
    "differentialAge",
    # "topic_x",
    # "groupMetadataID_x",
    # "signal.turnSignal",
    # "wheelSpeed.wheelSpdRr",
    # "wheelSpeed.wheelSpdFr",
    # "wheelSpeed.wheelSpdRl",
    # "wheelSpeed.wheelSpdFl",
    # "ChassisBestPoseMatchedTime",
    # "topic_y",
    # "gpsGlonassUsedMask",
    "time_y",
    # "longitudeStdDev",
    # "undulation",
    # "numSatsTracked",
    # "baseStationId",
    # "latitudeStdDev",
    # "solType",
    # "galileoBeidouUsedMask",
    # "numSatsL1",
    # "datumId",
    # "measurementTime",
    # "extendedSolutionStatus",
    # "heightStdDev",
    # "numSatsMulti",
    # "heightMsl",
    # "groupMetadataID_y",
    # "solutionAge",
    # "solStatus",
]


r_cols = [
    "BinaryDrivingMode"
    # "TernaryDrivingModeTransition"
]

In [ ]:
# Define your map with your dataframe name first and keep everything else the same
fig = px.scatter_mapbox(merged_df, 
                        lat="latitude",
                        lon="longitude",
                        hover_data=["time_x", "time_sec"],
                        #color="drivingMode",
                        zoom=13,
                        height=1000,
                        width=1200,
                        # color_discrete_map={
                        #   "M->A": "#63ACBE",
                        #   "A->M": "#601A4A",
                        #   "ERROR": "#EE442F",
                        #   "NONE":"yellowgreen",
                        # }
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }
                        #color_discrete_map={
                        #    "0": "#EE442F",
                        #    "1": "#63ACBE",
                        #}

                        )

# Layout settings
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.update_traces(marker=dict(size=8,opacity=1))

# Display the map
fig.show()

### Segmentation

In [ ]:
#i = merged_df[(merged_df["BinaryDrivingMode"] == 0) & (1 == merged_df.shift()["BinaryDrivingMode"])].index[0]

delta_time = 10000000000 # 10 seconds
#delta_time = 60000000000 # 60 seconds

i = 0

segments = pd.DataFrame()

for t in merged_df[(merged_df["TernaryDrivingModeTransition"] == -1)]["time_y"]:
    #print(t)

    segment =  merged_df[(merged_df["time_y"].between(t-delta_time,t))][d2_cols]
    
    segment["segment_num"] = i
    
    segments = pd.concat([segments, segment])
    i+= 1

segments    

In [ ]:
# Define your map with your dataframe name first and keep everything else the same
fig = px.scatter_mapbox(segments.drop_duplicates(subset=["latitude", "longitude"]), 
                        lat="latitude",
                        lon="longitude",
                        hover_data=d2_cols + ["segment_num"],
                        color="drivingMode",
                        zoom=13,
                        height=1000,
                        width=1200,
                        opacity=0.5,
                        # color_discrete_map={
                        #   "M->A": "#63ACBE",
                        #   "A->M": "#601A4A",
                        #   "ERROR": "#EE442F",
                        #   "NONE":"yellowgreen",
                        # }
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }
                        #color_discrete_map={
                        #    "0": "#EE442F",
                        #    "1": "#63ACBE",
                        #}

                        )

# Layout settings
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.update_traces(marker=dict(size=15))#, opacity=0.2))

# Display the map
fig.show()